In [2]:
import cv2
import numpy as np
from PIL import ImageGrab

In [4]:
def motion_detector_gist():

    previous_frame = None

    while True:
        # 1. Load image; convert to RGB
        img_brg = np.array(ImageGrab.grab())
        img_rgb = cv2.cvtColor(src=img_brg, code=cv2.COLOR_BGR2RGB)


        # 2. Prepare image; grayscale and blur
        prepared_frame = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
        prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5, 5), sigmaX=0)

        # 2. Calculate the difference
        if (previous_frame is None):
          # First frame; there is no previous one yet
            previous_frame = prepared_frame
            continue

        # 3. calculate difference and update previous frame
        diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
        previous_frame = prepared_frame

        # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
        kernel = np.ones((5, 5))
        diff_frame = cv2.dilate(diff_frame, kernel, 1)

        # 5. Only take different areas that are different enough (>20 / 255)
        thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

        # 6. Find and optionally draw contours
        contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
        # Comment below to stop drawing contours
        cv2.drawContours(image=img_rgb, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        # Uncomment 6 lines below to stop drawing rectangles
        # for contour in contours:
        #   if cv2.contourArea(contour) < 50:
        #     # too small: skip!
        #       continue
        #   (x, y, w, h) = cv2.boundingRect(contour)
        #   cv2.rectangle(img=img_rgb, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)

        cv2.imshow('Motion detector', img_rgb)

        if (cv2.waitKey(30) == 27):
          # out.release()
          break

        # Cleanup
        cv2.destroyAllWindows()

In [1]:
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2
import numpy as np

In [2]:
positive = "/mnt/projects/PPM/F387-1.h264"
negative = "/mnt/projects/PPM/Monitor_220425_Pi4_124013.h264"

In [6]:
cap= cv2.VideoCapture(positive)
firstFrame = None
motion = 0

while(cap.isOpened()):
    
    ret, frame = cap.read()
    
    if frame is None:
        break
        
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if firstFrame is None:
        firstFrame = gray
        continue
        
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    motion += np.average(thresh)

    cv2.imshow('frame',thresh)
    cv2.imshow('frame',thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
print(motion)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to target thread (0x564ee9f97500)

QObject::moveToThread: Current thread (0x564ee9f97500) is not the object's thread (0x564ee9b50160).
Cannot move to tar

59462.41782493364


In [13]:
# loop over the frames of the video
while True:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = vs.read()
    frame = frame[1]
    text = "Unoccupied"
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < args["min_area"]:
            continue
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Occupied"
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
    # cleanup the camera and close any open windows
    vs.release()
    cv2.destroyAllWindows()

NameError: name 'vs' is not defined

In [ ]:
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < args["min_area"]:
            continue
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Occupied"
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)